In [0]:

from pyspark.sql.functions import *
from pyspark.sql.functions import year
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")


Out[1]: DataFrame[key: string, value: string]

In [0]:
amazon_df = spark.read.csv("dbfs:/FileStore/tables/all_transaction_1-1.csv",header = True , inferSchema = True)

In [0]:
print(type(amazon_df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [0]:
amazon_df1 = amazon_df.withColumn("Year",year(amazon_df.InvoiceDate)).withColumn("Month",month(amazon_df.InvoiceDate)).withColumn("Date",to_date((amazon_df.InvoiceDate),"MM/dd/yyyy"))

In [0]:
print(type(amazon_df1))

<class 'pyspark.sql.dataframe.DataFrame'>


In [0]:
amazon_df1.createOrReplaceTempView("amazon_df_table")

In [0]:
amazon_df1.show()

+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+----------+
|         ProductCode|         Description|       Brand|Category|        SubCategory|ActualPrice|Marginpercentage|Discountpercentage|Quantity|TransactionAmount|        InvoiceDate|          Mode|InvoiceNumber|  CustomerId|  Country|Year|Month|      Date|
+--------------------+--------------------+------------+--------+-------------------+-----------+----------------+------------------+--------+-----------------+-------------------+--------------+-------------+------------+---------+----+-----+----------+
|e6fa885e-5f14-4e8...|Landau Pants 2xl ...|      landau|   shoes|  all women's shoes|     1470.0|              20|                30|       3|           3087.0|2015-01-04 18:30:00|    mastercard|   1045713097|GgZx5HTWOcva|Hong Kong|201

##1.	The particular inventory you need to store at a specific location

In [0]:
# a. a.	Find the top five performing mobile brands for the months of November, December and January  each year.
spark.sql("select Brand ,Month, year, count(*) as noofmobilebrands from amazon_df_table  where Month in (11,12,1) and SubCategory = 'mobile' group by  year , Brand,Month order by noofmobilebrands desc  ").show(5)

+-------+-----+----+----------------+
|  Brand|Month|year|noofmobilebrands|
+-------+-----+----+----------------+
|samsung|   12|2015|             208|
|samsung|   11|2015|             183|
|samsung|    1|2018|             166|
|samsung|   11|2016|             153|
|samsung|    1|2017|             153|
+-------+-----+----+----------------+
only showing top 5 rows



In [0]:
# b.	Find the top five selling products for the months of November, December and January for each year.

spark.sql("select SubCategory,sum(Quantity) as Total_sold , year from amazon_df_table where Month in (11,12,1) group by SubCategory ,year order by Total_sold desc").show(5)

+-----------------+----------+----+
|      SubCategory|Total_sold|year|
+-----------------+----------+----+
|all women's shoes|      8176|2017|
|all women's shoes|      7905|2015|
|           mobile|      7891|2015|
|           mobile|      6752|2016|
|      men's shoes|      6653|2016|
+-----------------+----------+----+
only showing top 5 rows



##2.	Give valuable insights to the investors

In [0]:
#a.	Number of products sold along with revenue aggregated quarterly
spark.sql("select Brand,sum(Quantity) as Num_Product_Sold , quarter(Date) as quater_revenue from amazon_df_table group by year,Brand,quater_revenue order by Num_Product_Sold desc").show()

+-------+----------------+--------------+
|  Brand|Num_Product_Sold|quater_revenue|
+-------+----------------+--------------+
|   nike|            2902|             1|
|   nike|            2743|             1|
|   nike|            2731|             2|
|   nike|            2729|             4|
|   nike|            2645|             4|
|   nike|            2642|             1|
|   nike|            2623|             1|
|   nike|            2544|             3|
|   nike|            2520|             3|
|   nike|            2520|             4|
|   nike|            2399|             2|
|   nike|            2336|             2|
|   nike|            2330|             3|
|samsung|            1536|             4|
|samsung|            1463|             2|
|samsung|            1434|             3|
|samsung|            1349|             3|
|samsung|            1334|             4|
|samsung|            1302|             1|
|samsung|            1237|             1|
+-------+----------------+--------

In [0]:
# b.	Number of products sold in each category aggregated quarterly
spark.sql("select Category,Brand , quarter(Date) as quamonth,count(Quantity) as NU_of_Category from amazon_df_table group by Category ,Date , Brand order by quamonth desc").show()

+-----------+------------+--------+--------------+
|   Category|       Brand|quamonth|NU_of_Category|
+-----------+------------+--------+--------------+
|   clothing|      donald|       4|             2|
|   clothing|    converse|       4|             1|
|      shoes|        sexy|       4|             1|
|electronics|       leeco|       4|             1|
|   clothing|     colored|       4|             1|
|electronics|       epson|       4|             1|
|   clothing|      simply|       4|             1|
|   clothing|    creative|       4|             1|
|   clothing|   chokushop|       4|             1|
|electronics|  blackberry|       4|             5|
|   clothing|        fila|       4|             1|
|   clothing|      bryant|       4|             1|
|electronics|     pandaoo|       4|             2|
|   clothing|    stafford|       4|             1|
|electronics|    logitech|       4|             1|
|   clothing|    luichiny|       4|             1|
|   clothing|      smiley|     

In [0]:
# c.	A number of products sold for each brand aggregated quarterly.
spark.sql("select Brand,year,quarter(Date),Count(Quantity) as Num_Product_Sold from amazon_df_table group by year,Brand, Date order by Num_Product_Sold desc").show()

+-----+----+-------------+----------------+
|Brand|year|quarter(Date)|Num_Product_Sold|
+-----+----+-------------+----------------+
| nike|2016|            4|              60|
| nike|2017|            4|              54|
| nike|2015|            4|              53|
| nike|2018|            1|              53|
| nike|2015|            1|              52|
| nike|2016|            1|              52|
| nike|2017|            1|              52|
| nike|2015|            3|              51|
| nike|2017|            2|              51|
| nike|2017|            2|              51|
| nike|2016|            3|              51|
| nike|2015|            4|              51|
| nike|2015|            1|              51|
| nike|2015|            1|              50|
| nike|2016|            4|              50|
| nike|2016|            2|              50|
| nike|2016|            4|              50|
| nike|2015|            2|              49|
| nike|2016|            3|              49|
| nike|2015|            3|      

##3.	Find products that generated profit

In [0]:
#a.	Top 5 profit margin generating products for a given month.
spark.sql("select SubCategory,Marginpercentage,Month,Year from amazon_df_table group by SubCategory,Marginpercentage,Month,Year order by Marginpercentage desc").show()



+--------------------+----------------+-----+----+
|         SubCategory|Marginpercentage|Month|Year|
+--------------------+----------------+-----+----+
|see more under ar...|              30|    4|2015|
|             walking|              30|    2|2015|
|     shoes & jewelry|              30|    5|2015|
|      women's skirts|              30|    4|2015|
|           moccasins|              30|    5|2015|
|      men's clothing|              30|    4|2015|
|see more samsung ...|              30|    5|2015|
|women's coats & j...|              30|    1|2015|
|               women|              30|    6|2015|
|            slippers|              30|    4|2015|
|             apparel|              30|    6|2015|
|  women's sunglasses|              30|    3|2015|
|              skirts|              30|    2|2015|
|       lawn & garden|              30|    2|2015|
|               youth|              30|    3|2015|
|                bras|              30|    5|2015|
|women's anniversa...|         

#4.	Brands that need marketing

In [0]:
# a.	Brand wise segregation of best performing categories profits wise
spark.sql("select Brand,Category,(Marginpercentage-Discountpercentage) as profit from amazon_df_table group by Brand,Category,(Marginpercentage-Discountpercentage) order by profit desc").show()

+--------------------+-----------+------+
|               Brand|   Category|profit|
+--------------------+-----------+------+
|               peter|   clothing|    20|
|             kenwood|electronics|    20|
|            castelli|   clothing|    20|
|                 dvs|   clothing|    20|
|               annie|   clothing|    20|
|              saddle|   clothing|    20|
|          underwraps|   clothing|    20|
|           haflinger|   clothing|    20|
|                 jbu|   clothing|    20|
|            muckboot|   clothing|    20|
|            siriusxm|electronics|    20|
|            funtasma|   clothing|    20|
|               perry|   clothing|    20|
|               donna|      shoes|    20|
|              noriko|   clothing|    20|
|                tumi|electronics|    20|
|                 cet|   clothing|    20|
|               nicki|      shoes|    20|
|yu&yu soccer men'...|   clothing|    20|
|               saint|   clothing|    20|
+--------------------+-----------+

In [0]:
# b.	Brand wise segregation of least performing categories orders wise
spark.sql("select Brand,Category,(Marginpercentage-Discountpercentage) as profit from amazon_df_table group by Brand,Category,(Marginpercentage-Discountpercentage) order by profit ").show()



+--------------------+-----------+------+
|               Brand|   Category|profit|
+--------------------+-----------+------+
|           alexander|   clothing|   -50|
|               ridge|   clothing|   -50|
|               bgood|   clothing|   -50|
|             pantech|electronics|   -50|
|            alliance|   clothing|   -50|
|             bellini|   clothing|   -50|
|             natural|   clothing|   -50|
|         ellie shoes|   clothing|   -50|
|                five|   clothing|   -50|
|fruit of the loom...|   clothing|   -50|
|                easy|   clothing|   -50|
|              report|   clothing|   -50|
|                lowa|   clothing|   -50|
|                 nwt|   clothing|   -50|
|             emporio|   clothing|   -50|
|            siriusxm|electronics|   -50|
|            onitsuka|   clothing|   -50|
|             dickies|   clothing|   -50|
|               bally|   clothing|   -50|
|                moda|   clothing|   -50|
+--------------------+-----------+

#5.	Find the products that should be part of the lighting deals.

In [0]:
#a.	Find the top five selling products in a given month
spark.sql("select SubCategory,sum(Quantity) as Total_sold from amazon_df_table group by SubCategory order by Total_sold desc").show(5)

+-----------------+----------+
|      SubCategory|Total_sold|
+-----------------+----------+
|all women's shoes|     92917|
|           mobile|     87843|
|      men's shoes|     79308|
|  shoes & jewelry|     33735|
|    women's shoes|     27424|
+-----------------+----------+
only showing top 5 rows



In [0]:
#b.	Find the top five products having the highest margin in a given month
spark.sql("select SubCategory,Marginpercentage,Month from amazon_df_table group by SubCategory,Marginpercentage,Month order by Marginpercentage desc").show()

+--------------------+----------------+-----+
|         SubCategory|Marginpercentage|Month|
+--------------------+----------------+-----+
|               rings|              30|    4|
|    juniors swimwear|              30|    4|
|          headphones|              30|    4|
|    medical uniforms|              30|    1|
|women's coats & j...|              30|    1|
|   intimates & sleep|              30|    4|
|               shoes|              30|    3|
|women's casual shoes|              30|    5|
|       women's boots|              30|    3|
|           religious|              30|    3|
|           computers|              30|    5|
|            athletic|              30|    1|
|intimates & sleep...|              30|    2|
|            swimwear|              30|    2|
|       running shoes|              30|    2|
|               rings|              30|    3|
|               youth|              30|    3|
|         men's rings|              30|    2|
|         golf gloves|            

In [0]:
# c.	Find the five lowest selling products in a given month.
spark.sql("select SubCategory,Marginpercentage,Month from amazon_df_table group by SubCategory,Marginpercentage,Month order by Marginpercentage ").show()

+--------------------+----------------+-----+
|         SubCategory|Marginpercentage|Month|
+--------------------+----------------+-----+
|      sporting goods|             -20|    5|
|    shoes & handbags|             -20|    3|
|      men's clothing|             -20|    5|
|         electronics|             -20|    2|
|sunglasses & eyew...|             -20|    5|
|               women|             -20|    2|
|see more samsung ...|             -20|    6|
|  bags & accessories|             -20|    4|
|sunglasses & eyew...|             -20|    6|
|     women's sandals|             -20|    5|
|  women's sunglasses|             -20|    1|
|      sporting goods|             -20|    3|
| athletic supporters|             -20|    2|
|               men's|             -20|    2|
|       wrist watches|             -20|    1|
|samsung tax time ...|             -20|    6|
|              mobile|             -20|    2|
|portable bluetoot...|             -20|    3|
|              office|            

##6.	Trending brands for given month order wise

In [0]:
spark.sql("select Brand ,MOnth,count(Month) as Count_of_brands from amazon_df_table  group by Brand ,Month order by Count_of_brands desc ").show()

+-------+-----+---------------+
|  Brand|MOnth|Count_of_brands|
+-------+-----+---------------+
|   nike|    3|           1601|
|   nike|    2|           1340|
|   nike|    1|           1325|
|   nike|   11|           1163|
|   nike|    5|           1079|
|   nike|    4|           1036|
|   nike|    8|           1033|
|   nike|    9|           1012|
|   nike|   12|           1002|
|   nike|   10|            913|
|   nike|    7|            889|
|   nike|    6|            854|
|samsung|    3|            735|
|samsung|    1|            655|
|samsung|    4|            605|
|samsung|    8|            587|
|samsung|   10|            553|
|samsung|    2|            530|
|samsung|   11|            530|
|samsung|   12|            509|
+-------+-----+---------------+
only showing top 20 rows

